## code 1

In [1]:
import time
import os
import sys

import pybullet as p

from cairo_simulator.core.utils import ASSETS_PATH
from cairo_simulator.core.log import Logger
from cairo_simulator.core.simulator import Simulator, SimObject
from cairo_simulator.devices.manipulators import Sawyer
from utils import load_configuration, save_config_to_configuration_file, manual_control, create_cuboid_obstacle

from pybullet_utils import bullet_client as bc
from pybullet_utils import urdfEditor as ed
import pybullet_data
import time



def main():

    use_ros = False
    use_real_time = True
    logger = Logger()
    sim = Simulator(logger=logger, use_ros=use_ros, use_real_time=use_real_time) # Initialize the Simulator
    p.setGravity(0,0,-9.81)
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,0)
    p.setPhysicsEngineParameter(enableFileCaching=0)
    ground_plane = SimObject("Ground", "plane.urdf", [0,0,0])
    # Add a table and a Sawyer robot
    #table = SimObject("Table", ASSETS_PATH + 'table.sdf', (0.9, 0, 0), (0, 0, 1.5708),fixed_base=1) #rotated 90deg (in radians)along z-axis
    
    table = SimObject('table', 'NEWtable.urdf',  (0.9, 0.1, .47),(1.5708*2,0,0),fixed_base=1)

    #print(p.getNumJoints(table.get_simulator_id()))
    
    
    sawyer_robot = Sawyer("sawyer0", [0, 0, 0.8], fixed_base=1)
    #print(p.getNumJoints(sawyer_robot))
    
    
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
    
    block = SimObject('block1', '1block.urdf', (0.69, -0.2, .55))
    sim_obj1 = SimObject('hole1', '1.5hole.urdf',  (0.69, 0.1, .530),(0,0,0),fixed_base=0)  #1.5708 for 90 deg rotation
    sim_obj2 = SimObject('hole2', '1.25hole.urdf', (0.69, 0.3, .530),(0,0,0),fixed_base=0)
    sim_obj3 = SimObject('hole3', '1.15hole.urdf', (0.69, 0.5, .530),(0,0,0),fixed_base=0)    
    
    #___________________#Combining URDF's
    #https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/combineUrdf.py
    p0 = bc.BulletClient(connection_mode=p.DIRECT)
    p0.setAdditionalSearchPath(pybullet_data.getDataPath())
    p1 = bc.BulletClient(connection_mode=p.DIRECT)
    p1.setAdditionalSearchPath(pybullet_data.getDataPath())

    #husky = p1.loadURDF('1block.urdf', flags=p0.URDF_USE_IMPLICIT_CYLINDER)  #this flag appears to have no impact
    husky = p1.loadURDF('1block.urdf')
    kuka = p0.loadURDF('1.5hole.urdf')

    ed0 = ed.UrdfEditor()
    ed0.initializeFromBulletBody(husky, p1._client)
    ed1 = ed.UrdfEditor()
    ed1.initializeFromBulletBody(kuka, p0._client)

    parentLinkIndex = 0

    jointPivotXYZInParent = [0, 0, 0]
    jointPivotRPYInParent = [0, 0, 0]

    jointPivotXYZInChild = [0, 0, 0]
    jointPivotRPYInChild = [0, 0, 0]

    newjoint = ed0.joinUrdf(ed1, parentLinkIndex, jointPivotXYZInParent, jointPivotRPYInParent,
                            jointPivotXYZInChild, jointPivotRPYInChild, p0._client, p1._client)
    newjoint.joint_type = p0.JOINT_FIXED

    ed0.saveUrdf("combined.urdf")
    #pgui = bc.BulletClient(connection_mode=p.GUI)
    #pgui.configureDebugVisualizer(pgui.COV_ENABLE_RENDERING, 0)
    
    orn = [0, 0, 0, 1]
    #ed0.createMultiBody([0, 0, 0], orn, pgui._client)  #doesn't work
    
    
    #ed0.createMultiBody([(0.69, -0.1, .530)], orn)  #works
    sim_obj3 = SimObject('hole3',"combined.urdf", (0.69, -0.4, .530),(0,0,0),fixed_base=0)  
    #_______________________
    
    orientation= [0,1,0,0] #quaternion hand pointing down..ish
    xoffset=0
    yoffset=0
    
    joint_config2 = [0.69+xoffset, 0.1+yoffset, 1]
    joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
    sawyer_robot.move_to_joint_pos(joint_config)
    time.sleep(1.)
    joint_config2 = [0.69+xoffset, 0.1+yoffset, 0.8]
    joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
    sawyer_robot.move_to_joint_pos(joint_config)
    
    key_w = ord('i') 
    key_a = ord('j') 
    key_s = ord('k') 
    key_d = ord('l') 
    key_8 = ord('8') 
    
    loop_count = 0 # Loop until someone shuts us down
    h=0.8
    xcurrent=0.69
    ycurrent=0.1
    # the center of 1.5 hole is roughly 0.71  0.08 OR   ~0.78 in x and y
    
    while True:
            xoffset=0
            yoffset=0
            
            keys = p.getKeyboardEvents()
            if key_w in keys and keys[key_w] & p.KEY_WAS_TRIGGERED:
                ycurrent+=0.01
                print(xcurrent,"  ",ycurrent)
                joint_configuration = [xcurrent, ycurrent, h]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_a in keys and keys[key_a] & p.KEY_WAS_TRIGGERED:
                xcurrent-=0.01
                print(xcurrent,"  ",ycurrent)
                joint_configuration = [xcurrent, ycurrent, h]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_s in keys and keys[key_s] & p.KEY_WAS_TRIGGERED:
                ycurrent-=0.01
                print(xcurrent,"  ",ycurrent)
                joint_configuration = [xcurrent, ycurrent, h]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_d in keys and keys[key_d] & p.KEY_WAS_TRIGGERED:
                xcurrent+=0.01
                print(xcurrent,"  ",ycurrent)
                joint_configuration = [xcurrent, ycurrent, h]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_8 in keys and keys[key_8] & p.KEY_WAS_TRIGGERED:
                print(p.getJointInfo(sawyer_robot.right_j6))
               
                
                
            time.sleep(0.2)
    #manual_control(sawyer_robot)    
    try:
        while True:
            sim.step()
    except KeyboardInterrupt:
        p.disconnect()
        sys.exit(0)

ModuleNotFoundError: No module named 'utils'

In [2]:
main()


NameError: name 'main' is not defined

In [ ]:
#https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/combineUrdf.py

from pybullet_utils import bullet_client as bc
from pybullet_utils import urdfEditor as ed
import pybullet
import pybullet_data
import time

p0 = bc.BulletClient(connection_mode=pybullet.DIRECT)
p0.setAdditionalSearchPath(pybullet_data.getDataPath())
p1 = bc.BulletClient(connection_mode=pybullet.DIRECT)
p1.setAdditionalSearchPath(pybullet_data.getDataPath())

husky = p1.loadURDF('1block.urdf', flags=p0.URDF_USE_IMPLICIT_CYLINDER)
kuka = p0.loadURDF('1.5hole.urdf')

ed0 = ed.UrdfEditor()
ed0.initializeFromBulletBody(husky, p1._client)
ed1 = ed.UrdfEditor()
ed1.initializeFromBulletBody(kuka, p0._client)

parentLinkIndex = 0

jointPivotXYZInParent = [0, 0, 0]
jointPivotRPYInParent = [0, 0, 0]

jointPivotXYZInChild = [0, 0, 0]
jointPivotRPYInChild = [0, 0, 0]

newjoint = ed0.joinUrdf(ed1, parentLinkIndex, jointPivotXYZInParent, jointPivotRPYInParent,
                        jointPivotXYZInChild, jointPivotRPYInChild, p0._client, p1._client)
newjoint.joint_type = p0.JOINT_FIXED

ed0.saveUrdf("combined.urdf")

orn = [0, 0, 0, 1]
ed0.createMultiBody([0, 0, 0], orn, pgui._client)


## code 2


In [ ]:
#https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/combineUrdf.py

from pybullet_utils import bullet_client as bc
from pybullet_utils import urdfEditor as ed
import pybullet
import pybullet_data
import time

p0 = bc.BulletClient(connection_mode=pybullet.DIRECT)
p0.setAdditionalSearchPath(pybullet_data.getDataPath())

p1 = bc.BulletClient(connection_mode=pybullet.DIRECT)
p1.setAdditionalSearchPath(pybullet_data.getDataPath())

#can also connect using different modes, GUI, SHARED_MEMORY, TCP, UDP, SHARED_MEMORY_SERVER, GUI_SERVER

husky = p1.loadURDF('1block.urdf', flags=p0.URDF_USE_IMPLICIT_CYLINDER)
kuka = p0.loadURDF('1.5hole.urdf')


ed0 = ed.UrdfEditor()
ed0.initializeFromBulletBody(husky, p1._client)
ed1 = ed.UrdfEditor()
ed1.initializeFromBulletBody(kuka, p0._client)

parentLinkIndex = 0

jointPivotXYZInParent = [.1, 0, 0]
jointPivotRPYInParent = [0, 0, 0]

jointPivotXYZInChild = [0, 0, 0]
jointPivotRPYInChild = [0, 0, 0]

newjoint = ed0.joinUrdf(ed1, parentLinkIndex, jointPivotXYZInParent, jointPivotRPYInParent,
                        jointPivotXYZInChild, jointPivotRPYInChild, p0._client, p1._client)
newjoint.joint_type = p0.JOINT_FIXED

ed0.saveUrdf("combined.urdf")

print(p0._client)
print(p1._client)
print("p0.getNumBodies()=", p0.getNumBodies())
print("p1.getNumBodies()=", p1.getNumBodies())

pgui = bc.BulletClient(connection_mode=pybullet.GUI)
pgui.configureDebugVisualizer(pgui.COV_ENABLE_RENDERING, 0)

orn = [0, 0, 0, 1]
ed0.createMultiBody([0, 0, 0], orn, pgui._client)
pgui.setRealTimeSimulation(1)

pgui.configureDebugVisualizer(pgui.COV_ENABLE_RENDERING, 1)

while (pgui.isConnected()):
  pgui.getCameraImage(320, 200, renderer=pgui.ER_BULLET_HARDWARE_OPENGL)
  time.sleep(1. / 240.)


0
1
p0.getNumBodies()= 1
p1.getNumBodies()= 1
fileNames= ['./1.5in squarehole_C.SLDPRT/meshes/1.5in squarehole_C.SLDPRT.STL']
